In [1]:
import numpy as np
import pandas as pd

from utils import *

## Load Data

In [3]:
# cfpb_df = pd.read_excel('./Real-data/PB_CF_to_SP_NKH.xlsx')
# cfpb_df.head()

## Data Check

In [4]:
# cfpb_df.isnull().sum()

In [5]:
# cfpb_df[cfpb_df['Mã SP cấp 5'].isnull()].head()

In [6]:
# cfpb_df[cfpb_df['Mã SP cấp 5'].isnull()]['Chi phí nhận phân bổ tại thời điểm'].sum()

In [7]:
# cfpb_df['Tên SP cấp 5'].value_counts()

In [8]:
# cfpb_df[cfpb_df['Mã SP cấp 5'] == 99].head()

In [9]:
# cfpb_df[cfpb_df['Mã SP cấp 5'].isnull()]['Chi phí nhận phân bổ tại thời điểm'].sum()

In [10]:
# cfpb_df[cfpb_df['Mã SP cấp 5'] == 99].head()

In [11]:
# cfpb_df[cfpb_df['Mã SP cấp 5'] == 21020499].head()

## Data preprocessing

In [12]:
# cfpb_df['Mã SP cấp 5'].fillna(99, inplace= True)
# cfpb_df['Mã SP cấp 5'] = cfpb_df['Mã SP cấp 5'].astype('int64')

# cfpb_df['Tên SP cấp 5'].fillna('SAN PHAM KHAC', inplace= True)

In [13]:
# cfpb_df['Tháng'] = pd.to_datetime(cfpb_df['Tháng']).dt.strftime('%Y/%m')

In [14]:
# cfpb_df.to_parquet('./Real-data/PB_CF_to_SP_NKH.parquet')

In [15]:
cfpb_df = pd.read_parquet('./Real-data/PB_CF_to_SP_NKH.parquet')

In [16]:
map_org_cost_type = pd.read_csv('./Real-data/Phân loại cost type và CP Phân loại phòng - Cost ID Type.csv')
map_org_cost_type.head()

,Driver_type,FROM_ORG_UNIT_ID,FROM_COST_TYPE_ID,TO_COST_TYPE_ID,TO_ORG_UNIT_ID,VALUE,V_MAKER_REMARKS
0,CPTTPB,7199,99,9910,117200071000,0.37,NaN
1,CPTTPB,7199,99,9910,117400071000,0.63,NaN
2,CPTTPB,117000071000,99,9910,117200071000,0.37,NaN
3,CPTTPB,117000071000,99,9910,117400071000,0.63,NaN
4,CPTTPB,117000071764,99,9910,117200071000,0.37,NaN


In [17]:
map_cost_type = pd.read_csv('./Real-data/Phân loại cost type và CP Phân loại phòng - Phân Loại Cost Type.csv')
map_cost_type.head()

,Tên Cost_type,Mã Cost_type,Unnamed: 2
0,CHI PHI HO TRO HOAT DONG,9901,Hỗ trợ hoạt động
1,CHI PHI QUAN HE KHACH HANG,9902,QHKH
2,CHI PHI QUAN LY SAN PHAM,9903,QLSP
3,CHI PHI KINH DOANH,9904,Kinh doanh
4,CHI PHI XU LY GIAO DICH,9905,XL Giao dich


In [18]:
map_org_to_cost_type = map_org_cost_type.set_index(['TO_ORG_UNIT_ID'])['TO_COST_TYPE_ID'].to_dict()
map_cost_type_name = map_cost_type.set_index(['Mã Cost_type'])['Tên Cost_type'].to_dict()

In [19]:
map_org_to_cost_type

{117200071000: 9910,
 117400071000: 9910,
 117500072000: 9908,
 117500073000: 9908,
 117500131000: 9908,
 99099: 9908,
 99199: 9908,
 99699: 9908,
 115500998460: 9908,
 117500074000: 9909,
 114200996340: 9906,
 117500996652: 9908,
 117500996651: 9908,
 111500996040: 9908,
 112500996180: 9908,
 122500996223: 9908,
 124200996310: 9908,
 124200996311: 9908,
 115500996430: 9908,
 115500996460: 9908,
 116500996510: 9908,
 116500996530: 9908,
 118200996530: 9908,
 116500996540: 9908,
 116500996560: 9908,
 117500996601: 9908,
 117500996620: 9908,
 117500996640: 9908,
 117500996670: 9908,
 117500996730: 9908,
 117500996790: 9908,
 117500996791: 9908,
 117500996792: 9908,
 119500996818: 9908,
 121500996830: 9908,
 121500996831: 9908,
 119500996838: 9908,
 123500996840: 9908,
 123500996841: 9908,
 123500996842: 9908,
 123500996843: 9908,
 123500996844: 9908,
 123500996845: 9908,
 119500996848: 9908,
 119500996858: 9908,
 119500996868: 9908,
 119500996878: 9908,
 119500996888: 9908,
 120500996910

In [20]:
map_cost_type_name

{9901: 'CHI PHI HO TRO HOAT DONG',
 9902: 'CHI PHI QUAN HE KHACH HANG',
 9903: 'CHI PHI QUAN LY SAN PHAM',
 9904: 'CHI PHI KINH DOANH',
 9905: 'CHI PHI XU LY GIAO DICH',
 9906: 'CHI PHI QUAN LY RUI RO',
 9907: 'CHI PHI XAY DUNG NANG LUC',
 9908: 'CHI PHI DUY TRI HOAT DONG',
 9909: 'CHI PHI DICH VU NOI BO',
 9910: 'CHI PHI CONG NGHE THONG TIN'}

In [21]:
cfpb_df['COST_TYPE'] = cfpb_df['Mã đơn vị tổ chức cấp 6'].map(map_org_to_cost_type)
cfpb_df['COST_TYPE_NAME'] = cfpb_df['COST_TYPE'].map(map_cost_type_name)
cfpb_df

,Tháng,Mã đơn vị tổ chức cấp 5,Tên đơn vị tổ chức cấp 5,Mã đơn vị tổ chức cấp 6,Tên đơn vị tổ chức cấp 6,Mã SP cấp 5,Tên SP cấp 5,Mã phân khúc KH cấp 3,Tên phân khúc KH cấp 3,Chi phí nhận phân bổ tại thời điểm,COST_TYPE,COST_TYPE_NAME
0,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20010501,CHO VAY THE TIN DUNG,22,KHACH HANG CA NHAN VIP,3.800000e-01,9902.0,CHI PHI QUAN HE KHACH HANG
1,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,11,KHACH HANG DOANH NGHIEP LON,4.333343e+07,9902.0,CHI PHI QUAN HE KHACH HANG
2,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,12,KHACH HANG DOANH NGHIEP VUA VA NHO (SMEs),2.035640e+08,9902.0,CHI PHI QUAN HE KHACH HANG
3,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,24,KHACH HANG DOANH NGHIEP QUY MO SIEU NHO (ME),1.533729e+05,9902.0,CHI PHI QUAN HE KHACH HANG
4,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030129,GOI TIN DUNG KHDN 2019,11,KHACH HANG DOANH NGHIEP LON,3.475100e+01,9902.0,CHI PHI QUAN HE KHACH HANG
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2023/12,323,CN DONG HAI PHONG,112100323154,PGD KIEN AN_CN DONG HAI PHONG,99,SAN PHAM KHAC,99,UNALLOCABLE,0.000000e+00,9902.0,CHI PHI QUAN HE KHACH HANG
999996,2023/12,323,CN DONG HAI PHONG,112100323155,PGD DINH VU_CN DONG HAI PHONG,20010102,CHO VAY MUA OTO TIEU DUNG,21,KHACH HANG CA NHAN THONG THUONG,1.718514e+03,9902.0,CHI PHI QUAN HE KHACH HANG
999997,2023/12,323,CN DONG HAI PHONG,112100323155,PGD DINH VU_CN DONG HAI PHONG,20010201,CHO VAY MUA NHA THONG THUONG,21,KHACH HANG CA NHAN THONG THUONG,6.787703e+03,9902.0,CHI PHI QUAN HE KHACH HANG
999998,2023/12,323,CN DONG HAI PHONG,112100323155,PGD DINH VU_CN DONG HAI PHONG,20010201,CHO VAY MUA NHA THONG THUONG,22,KHACH HANG CA NHAN VIP,2.101592e+04,9902.0,CHI PHI QUAN HE KHACH HANG


In [22]:
map_df = pd.read_csv('Map_V1.csv')
map_tt = map_df.set_index('LEVEL_06_CODE')['LEVEL_05_NAME'].to_dict()
map_khoi = map_df.set_index('LEVEL_06_CODE')['LEVEL_KHOI_NAME'].to_dict()
map_tt_color = map_df.set_index('LEVEL_05_NAME')['LEVEL_TT_COLOR'].to_dict()
map_khoi_color = map_df.set_index('LEVEL_KHOI_NAME')['LEVEL_KHOI_COLOR'].to_dict()

In [30]:
map_sp_df = pd.read_excel('./Real-data/2023-12-13 Cây phân cấp phục vụ phân bổ Sanky Chart.xlsx', 
                          sheet_name= 'Product_Hierarchy final to FIS')
map_sp = map_sp_df.set_index('PRODUCT_LEAF_ID')['LEVEL2_DESC'].to_dict()

In [31]:
cfpb_df['Mã đơn vị tổ chức cấp 6_tt'] = cfpb_df['Mã đơn vị tổ chức cấp 6'].map(map_tt)
cfpb_df['Mã đơn vị tổ chức cấp 6_khoi'] = cfpb_df['Mã đơn vị tổ chức cấp 6'].map(map_khoi)
cfpb_df['Mã SP cấp 5 PK'] = cfpb_df['Mã SP cấp 5'].map(map_sp)

In [32]:
selected_columns = ['COST_TYPE_NAME', 'Mã SP cấp 5 PK', 'Tên phân khúc KH cấp 3']
pk_df = cfpb_df.groupby(selected_columns).agg({'Chi phí nhận phân bổ tại thời điểm': 'sum'}).reset_index()
pk_df

,COST_TYPE_NAME,Mã SP cấp 5 PK,Tên phân khúc KH cấp 3,Chi phí nhận phân bổ tại thời điểm
0,CHI PHI CONG NGHE THONG TIN,BAO LANH VA CAM KET TIN DUNG,KHACH HANG CA NHAN THONG THUONG,2.266350e+07
1,CHI PHI CONG NGHE THONG TIN,BAO LANH VA CAM KET TIN DUNG,KHACH HANG CA NHAN VIP,3.571730e+05
2,CHI PHI CONG NGHE THONG TIN,BAO LANH VA CAM KET TIN DUNG,KHACH HANG DINH CHE TAI CHINH,1.217447e+07
3,CHI PHI CONG NGHE THONG TIN,BAO LANH VA CAM KET TIN DUNG,KHACH HANG DOANH NGHIEP CO VON DAU TU NUOC NGO...,1.199984e+07
4,CHI PHI CONG NGHE THONG TIN,BAO LANH VA CAM KET TIN DUNG,KHACH HANG DOANH NGHIEP LON,6.799207e+07
...,...,...,...,...
443,CHI PHI XU LY GIAO DICH,TIEN GUI VA CAC CONG CU NO,KHACH HANG DOANH NGHIEP LON,1.537768e+10
444,CHI PHI XU LY GIAO DICH,TIEN GUI VA CAC CONG CU NO,KHACH HANG DOANH NGHIEP QUY MO SIEU NHO (ME),1.677992e+09
445,CHI PHI XU LY GIAO DICH,TIEN GUI VA CAC CONG CU NO,KHACH HANG DOANH NGHIEP VUA VA NHO (SMEs),3.724366e+10
446,CHI PHI XU LY GIAO DICH,TIEN GUI VA CAC CONG CU NO,KHACH HANG NGUON VON UY THAC QUOC TE,1.122812e+08


In [33]:
highlighted_nodes = []
fig = sankey_graph(pk_df, selected_columns, highlighted_nodes, 
                    title= "<b>Phân bổ giai đoạn 2: Từ CPQLKD trực tiếp đến PKKH</b>", 
                    size_column= 'Chi phí nhận phân bổ tại thời điểm', color_map= map_khoi_color)

In [3]:
cfpb_df = pd.read_parquet('./Real-data/PB_CF_to_SP_NKH.parquet')
map_org_cost_type = pd.read_csv('./Real-data/Phân loại cost type và CP Phân loại phòng - Cost ID Type.csv')
map_cost_type = pd.read_csv('./Real-data/Phân loại cost type và CP Phân loại phòng - Phân Loại Cost Type.csv')

map_org_to_cost_type = map_org_cost_type.set_index(['TO_ORG_UNIT_ID'])['TO_COST_TYPE_ID'].to_dict()
map_cost_type_name = map_cost_type.set_index(['Mã Cost_type'])['Tên Cost_type'].to_dict()

cfpb_df['COST_TYPE'] = cfpb_df['Mã đơn vị tổ chức cấp 6'].map(map_org_to_cost_type)
cfpb_df['COST_TYPE_NAME'] = cfpb_df['COST_TYPE'].map(map_cost_type_name)

In [5]:
cfpb_df.head()

,Tháng,Mã đơn vị tổ chức cấp 5,Tên đơn vị tổ chức cấp 5,Mã đơn vị tổ chức cấp 6,Tên đơn vị tổ chức cấp 6,Mã SP cấp 5,Tên SP cấp 5,Mã phân khúc KH cấp 3,Tên phân khúc KH cấp 3,Chi phí nhận phân bổ tại thời điểm,COST_TYPE,COST_TYPE_NAME
0,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20010501,CHO VAY THE TIN DUNG,22,KHACH HANG CA NHAN VIP,3.800000e-01,9902.0,CHI PHI QUAN HE KHACH HANG
1,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,11,KHACH HANG DOANH NGHIEP LON,4.333343e+07,9902.0,CHI PHI QUAN HE KHACH HANG
2,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,12,KHACH HANG DOANH NGHIEP VUA VA NHO (SMEs),2.035640e+08,9902.0,CHI PHI QUAN HE KHACH HANG
3,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,24,KHACH HANG DOANH NGHIEP QUY MO SIEU NHO (ME),1.533729e+05,9902.0,CHI PHI QUAN HE KHACH HANG
4,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030129,GOI TIN DUNG KHDN 2019,11,KHACH HANG DOANH NGHIEP LON,3.475100e+01,9902.0,CHI PHI QUAN HE KHACH HANG


In [7]:
map_sp_df = pd.read_excel('./Real-data/2023-12-13 Cây phân cấp phục vụ phân bổ Sanky Chart.xlsx', 
                        sheet_name= 'Product_Hierarchy final to FIS')
map_sp = map_sp_df.set_index('PRODUCT_LEAF_ID')['LEVEL2_DESC'].to_dict()
cfpb_df['Mã SP cấp 5 PK'] = cfpb_df['Mã SP cấp 5'].map(map_sp)
cfpb_df.head()

,Tháng,Mã đơn vị tổ chức cấp 5,Tên đơn vị tổ chức cấp 5,Mã đơn vị tổ chức cấp 6,Tên đơn vị tổ chức cấp 6,Mã SP cấp 5,Tên SP cấp 5,Mã phân khúc KH cấp 3,Tên phân khúc KH cấp 3,Chi phí nhận phân bổ tại thời điểm,COST_TYPE,COST_TYPE_NAME,Mã SP cấp 5 PK
0,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20010501,CHO VAY THE TIN DUNG,22,KHACH HANG CA NHAN VIP,3.800000e-01,9902.0,CHI PHI QUAN HE KHACH HANG,CHO VAY
1,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,11,KHACH HANG DOANH NGHIEP LON,4.333343e+07,9902.0,CHI PHI QUAN HE KHACH HANG,CHO VAY
2,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,12,KHACH HANG DOANH NGHIEP VUA VA NHO (SMEs),2.035640e+08,9902.0,CHI PHI QUAN HE KHACH HANG,CHO VAY
3,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030101,CHO VAY SXKD THONG THUONG,24,KHACH HANG DOANH NGHIEP QUY MO SIEU NHO (ME),1.533729e+05,9902.0,CHI PHI QUAN HE KHACH HANG,CHO VAY
4,2023/09,111,CN TRUNG TAM SAI GON,111100111021,PHONG KHACH HANG DOANH NGHIEP_CN TRUNG TAM SAI...,20030129,GOI TIN DUNG KHDN 2019,11,KHACH HANG DOANH NGHIEP LON,3.475100e+01,9902.0,CHI PHI QUAN HE KHACH HANG,CHO VAY


In [54]:
map_df = pd.read_csv('./Real-data/Phân loại cost type và CP Phân loại phòng - Phân Loại Phong.csv')
map_phong = map_df.set_index('LEVEL_06_CODE')['Ghi chú phân loại phòng'].to_dict()

cfpb_df['Mã đơn vị tổ chức cấp 6_phong'] = cfpb_df['Mã đơn vị tổ chức cấp 6'].map(map_phong)

selected_columns = ['COST_TYPE_NAME', 'Mã đơn vị tổ chức cấp 6_phong', 'Mã SP cấp 5 PK', 'Tên phân khúc KH cấp 3']
pk_df = cfpb_df.groupby(selected_columns).agg({'Chi phí nhận phân bổ tại thời điểm': 'sum'}).reset_index()

In [55]:
pk_df

,COST_TYPE_NAME,Mã đơn vị tổ chức cấp 6_phong,Mã SP cấp 5 PK,Tên phân khúc KH cấp 3,Chi phí nhận phân bổ tại thời điểm
0,CHI PHI HO TRO HOAT DONG,CHUNG,BAO LANH VA CAM KET TIN DUNG,KHACH HANG AO KHOI BAN BUON,1.266496e+07
1,CHI PHI HO TRO HOAT DONG,CHUNG,BAO LANH VA CAM KET TIN DUNG,KHACH HANG CA NHAN THAN THIET,4.367980e+07
2,CHI PHI HO TRO HOAT DONG,CHUNG,BAO LANH VA CAM KET TIN DUNG,KHACH HANG CA NHAN THONG THUONG,1.559887e+08
3,CHI PHI HO TRO HOAT DONG,CHUNG,BAO LANH VA CAM KET TIN DUNG,KHACH HANG CA NHAN VIP,2.003276e+08
4,CHI PHI HO TRO HOAT DONG,CHUNG,BAO LANH VA CAM KET TIN DUNG,KHACH HANG DINH CHE TAI CHINH,8.956731e+08
...,...,...,...,...,...
488,CHI PHI XU LY GIAO DICH,TTHTTT_PHONG QTTD,CHO VAY,KHACH HANG DOANH NGHIEP CO VON DAU TU NUOC NGO...,5.207158e+09
489,CHI PHI XU LY GIAO DICH,TTHTTT_PHONG QTTD,CHO VAY,KHACH HANG DOANH NGHIEP LON,5.347918e+10
490,CHI PHI XU LY GIAO DICH,TTHTTT_PHONG QTTD,CHO VAY,KHACH HANG DOANH NGHIEP QUY MO SIEU NHO (ME),1.490594e+09
491,CHI PHI XU LY GIAO DICH,TTHTTT_PHONG QTTD,CHO VAY,KHACH HANG DOANH NGHIEP VUA VA NHO (SMEs),4.011797e+10
